In [1]:
%%time
import sys;
import datetime
print(datetime.datetime.now())
print('---------------------------------------')
!{sys.executable} -m pip install -r requirements.txt


2020-11-18 14:54:29.517295
---------------------------------------
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
CPU times: user 72.8 ms, sys: 3.44 ms, total: 76.2 ms
Wall time: 2.31 s


In [2]:
%%time
import boto3
import sagemaker

session = sagemaker.Session() 
region = session.boto_region_name 

bucket = 'sagemaker-jfox'
 
prefix = 'sagemaker/xgboost'
role = sagemaker.get_execution_role()

print(f'Bucket {bucket} in region {region}')

Bucket sagemaker-jfox in region eu-west-1
CPU times: user 821 ms, sys: 68.1 ms, total: 889 ms
Wall time: 2.92 s


In [3]:
%%time
import random
import pandas as pd
raw_data_filename = 'frauddetection.csv'

s3 = boto3.resource('s3', region_name=region)
#s3.Bucket(bucket).download_file(raw_data_filename, raw_data_filename)
 

CPU times: user 62.8 ms, sys: 11.2 ms, total: 74 ms
Wall time: 1.08 s


---

## Evaluation

Now that we have our hosted endpoint, we can generate predictions from  the  test data set.

Compared actual to predicted values of whether the transaction was a "fraud" (`1`) or not (`0`).  Then we'll produce a  confusion matrix.

In [4]:
endpoint_name = 'fraud-detect-endpoint'

xgb_predictor = sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sagemaker.Session())

xgb_predictor.content_type = 'text/x-libsvm'
xgb_predictor.deserializer = None

def do_predict(data):
    payload = '\n'.join(data)
    response = xgb_predictor.predict(payload).decode('utf-8')
    result = response.split(',')
    preds = [float(num) for num in result]
    preds = [round(num) for num in preds]
    return preds

def batch_predict(data, batch_size):
    items = len(data)
    arrs = []
    
    for offset in range(0, items, batch_size):
        if offset+batch_size < items:
            results = do_predict(data[offset:(offset+batch_size)])
            arrs.extend(results)
        else:
            arrs.extend(do_predict(data[offset:items]))
        sys.stdout.write('.')
    return(arrs)

In [5]:
%%time
import json

with open('test.libsvm', 'r') as f:
    payload = f.read().strip()

labels = [int(line.split(' ')[0]) for line in payload.split('\n')]
test_data = [line for line in payload.split('\n')]
 
preds = batch_predict(test_data, 100)

.................CPU times: user 64.1 ms, sys: 239 µs, total: 64.3 ms
Wall time: 269 ms


In [6]:
print()
print(labels[30:60])
print(preds[30:60])


[0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1]


In [7]:
actual_pos = sum(1 for i in range(len(labels)) if 1 == labels[i])
actual_neg = sum(1 for i in range(len(labels)) if 0 == labels[i])

pred_pos = sum(1 for i in range(len(preds)) if 1 == preds[i])
pred_neg = sum(1 for i in range(len(preds)) if 0 == preds[i])

true_pos = sum(1 for i in range(len(preds)) if preds[i] == 1 == labels[i])
true_neg = sum(1 for i in range(len(preds)) if preds[i] == 0 == labels[i])

false_pos=sum(1 for i in range(len(preds)) if preds[i] == 1 and 0 == labels[i])
false_neg=sum(1 for i in range(len(preds)) if preds[i] == 0 and 1 == labels[i])

recall = true_pos / (true_pos + false_neg)
precision = true_pos / (true_pos + false_pos)
f1 = (2 * precision * recall) / (precision + recall)
error = sum(1 for i in range(len(preds)) if preds[i] != labels[i]) / float(len(preds))

assert recall <= f1 <= precision or precision <= f1 <= recall 
assert true_pos + false_neg == actual_pos
assert true_neg + false_pos == actual_neg
#print('pred_pos', pred_pos,'pred_neg', pred_neg)

assert len(preds)==len(labels)

#print(len(preds), 'predictions and labels')

#print('actual_pos', actual_pos, 'actual_neg', actual_neg)
assert actual_pos+actual_neg==len(labels)
                   
#print('true_pos', true_pos, 'false_pos', false_pos)
#assert  true_pos+false_pos == pred_pos

#print('true_neg', true_neg, 'false_neg', false_neg )
assert true_neg+false_neg== pred_neg

#print('pred_pos+pred_neg',pred_pos+pred_neg)
assert pred_pos+pred_neg==len(preds)

print(f'Recall={recall:.2f}')
print(f'Precision={precision:.2f}')
print(f'Error rate={error:.2f}')
print(f'F1={f1:.2f}')

Recall=1.00
Precision=0.95
Error rate=0.03
F1=0.97


Show confusion matrix.

In [9]:
import numpy as np
pd.crosstab(index=np.array(labels), columns=np.array(preds))

col_0,0,1
row_0,,
0,796,45
1,0,802


Clean up to save money

In [ ]:
#session.delete_endpoint(xgb_predictor.endpoint)